## Beaver Drought ET Data Visualization
### Written by Emily Fairfax in 2018
#### for educational and research purposes, displayed on emilyfairfaxscience.com

Use the following notebook to generate an interactive visualization of multiple rasters of ET data overlayed on a satellite image.

OR modify the code to make your own visualization for your own data!

In [22]:
#Import packages and functions
from bokeh.io import show
from bokeh.models import LinearColorMapper
from bokeh.palettes import Viridis256 as viridis
from bokeh.palettes import RdBu as rdbu
from bokeh.models.glyphs import ImageURL
from bokeh.plotting import figure
from bokeh.embed import components 
from bokeh.resources import CDN
from bokeh.models import HoverTool
import numpy as np
import pandas as pd

#set range of data rasters
x_range = 26
y_range = 25
total_cells = x_range * y_range

#set corners of rasters
base_xs = [0,1,1,0]
base_ys = [0,0,1,1]
grid_xs = []
grid_ys = []

#generate an empty grid of the necessary size
for h in range(y_range):
    for w in range(x_range):
        grid_xs.append([val+w for val in base_xs])
        grid_ys.append([val+h for val in base_ys])

        
# Overlay data
overlay_data = pd.read_csv('beaver_data.csv')
predrought_et = overlay_data['Pre-Drought ET'].tolist()
drought_et = overlay_data['Drought ET'].tolist()
et_diff = overlay_data['ET Difference'].tolist()
top_zone = overlay_data['Topographic Zone'].tolist()
beaver = overlay_data['Beaver?'].tolist()

In [30]:
#build data sets
predrought_data=dict(
    x=grid_xs, y=grid_ys,
    predrought_et=predrought_et, top_zone=top_zone, beaver=beaver
)

drought_data=dict(
    x=grid_xs, y=grid_ys,
    drought_et=drought_et, top_zone=top_zone, beaver=beaver
)


diff_data=dict(
    x=grid_xs, y=grid_ys,
    et_diff=et_diff, top_zone=top_zone, beaver=beaver
)

In [31]:
#set color maps
et_color_mapper = LinearColorMapper(palette=list(reversed(viridis)), low=0, high=8)
diff_color_mapper = LinearColorMapper(palette=list(reversed(rdbu[11])), low=-4, high=4)

In [32]:
#add tools to visualization
TOOLS = "pan,wheel_zoom,reset,hover,save"

#define the figure and add features
p = figure(
    title="Beaver Drought Buffering Data Visualization", 
    tools=TOOLS,
    x_axis_location=None, y_axis_location=None,
    x_range=(0, x_range), y_range=(0, y_range)
)

hover = HoverTool()
hover.tooltips = [ 
        ("Pre-Drought Evapotranspiration", "@predrought_et mm/day"),
        ("Drought Evapotranspiration", "@drought_et mm/day"),
        ("Evapotranspiration DIfference", "@et_diff mm/day"),
        ("Topographic Zone", "@top_zone"),
        ("Beaver ?", "@beaver")
    ]

p.tools.append(hover)
    

p.grid.grid_line_color = None
#p.hover.point_policy = "follow_mouse"

In [33]:
#pull background image
p.image_url(
    url=['https://i.imgur.com/MogWbug.jpg'],
    x=x_range / 2, y=y_range / 2, 
    w=x_range, h=y_range, 
    anchor="center"
)

#make patches (color rasters) for each data set
p.patches(
    'x', 'y', source=predrought_data,
    fill_color={'field': 'predrought_et', 'transform': et_color_mapper},
    fill_alpha=0.7, 
    line_color="white", 
    line_width=0.001,
    legend='Pre-Drought Evapotranspiration',
    visible=False
)

p.patches(
    'x', 'y', source=drought_data,
    fill_color={'field': 'drought_et', 'transform': et_color_mapper},
    fill_alpha=0.7, 
    line_color="white", 
    line_width=0.001,
    legend='Drought Evapotranspiration',
    visible=False
)

p.patches(
    'x', 'y', source=diff_data,
    fill_color={'field': 'et_diff', 'transform': diff_color_mapper},
    fill_alpha=0.7, 
    line_color="white", 
    line_width=0.001,
    legend='Evapotranspiration Difference',
    visible=False
)

#add the legend and legend interactivity
p.legend.location = "top_left"
p.legend.click_policy="hide"

In [34]:
#display figure
show(p)

# html = file_html(plot, CDN, "my plot")
# with open('full_html.html') as output_file:
#     output_file.write(html)

# INSTRUCTIONS FOR EMBEDDING IN WORDPRESS

Want to put a visualization like this on your own website? Follow the directions below!

1. uncomment the lines below and run it instead of the show(p) block
2. copy the div element out of the 'div.html' file into the post. It is already there but the id in the div might update when you recreate the plot
3. copy the code in the 'script.html' into the shortcode section.
4. put the shortcode link BEFORE the bokeh div.


In [35]:
 #script, div = components(p)
 #with open('script.html', 'w') as output_file:
 #    output_file.write(script)
 #with open('div.html', 'w') as output_file:
 #    output_file.write(div)